In [ ]:
# Update with your own model names

model_name_1 = 'sagemaker-xgboost-2020-06-09-08-33-24-782'
model_name_2 = 'sagemaker-xgboost-2020-02-24-14-41-35-744'

In [ ]:
production_variants = [
        {
            'VariantName': 'variant-1',
            'ModelName': model_name_1,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 9,
            'InstanceType': 'ml.t2.medium'
        },
        {
            'VariantName': 'variant-2',
            'ModelName': model_name_2,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 1,
            'InstanceType': 'ml.t2.medium'
        }
]

In [ ]:
import boto3
import pprint

sm = boto3.client('sagemaker')

In [ ]:
endpoint_config_name = 'xgboost-two-models-epc'

In [ ]:
response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants,
    Tags=[{'Key': 'Name','Value': endpoint_config_name},
          {'Key': 'Algorithm','Value': 'xgboost'}]
)
pprint.pprint(response)

In [ ]:
response = sm.describe_endpoint_config(EndpointConfigName=endpoint_config_name)

pprint.pprint(response)

In [ ]:
endpoint_name = 'xgboost-two-models-ep'

In [ ]:
response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    Tags=[{'Key': 'Name','Value': endpoint_name},
          {'Key': 'Algorithm','Value': 'xgboost'},
          {'Key': 'Environment','Value': 'development'}]
)

In [ ]:
response = sm.describe_endpoint(EndpointName=endpoint_name)

pprint.pprint(response)

In [ ]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98'

In [ ]:
smrt = boto3.Session().client(service_name='runtime.sagemaker') 

# Send to variants according to weights
response = smrt.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

In [ ]:
variants = ['variant-1', 'variant-2']

for v in variants:
    response = smrt.invoke_endpoint(EndpointName=endpoint_name, 
                                    ContentType='text/csv', 
                                    Body=test_sample,
                                    TargetVariant=v)
    print(response['Body'].read())


In [ ]:
response = sm.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        { 'VariantName': 'variant-1', 'DesiredWeight': 1},
        { 'VariantName': 'variant-2', 'DesiredWeight': 1}
    ]
)

In [ ]:
production_variants_2 = [
        {
            'VariantName': 'variant-2',
            'ModelName': model_name_2,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 1,
            'InstanceType': 'ml.t2.medium'
        }
]

endpoint_config_name_2 = 'xgboost-one-model-epc'

response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name_2,
    ProductionVariants=production_variants_2,
    Tags=[{'Key': 'Name','Value': endpoint_config_name_2},
          {'Key': 'Algorithm','Value': 'xgboost'}]
)

response = sm.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name_2
)

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name_2)